<a href="https://colab.research.google.com/github/hcbagde17/gdg_group_task1/blob/main/Copy_of_ML1_gdg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

In [6]:
import kagglehub

path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'ecommerce-dataset' dataset.
Path to dataset files: /kaggle/input/ecommerce-dataset


In [7]:
import pandas as pd

df_events = pd.read_csv(f'{path}/events.csv')
display(df_events.head())

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [8]:
filepath_part1 = f'{path}/item_properties_part1.csv'
filepath_part2 = f'{path}/item_properties_part2.csv'

df_item_properties_part1 = pd.read_csv(filepath_part1, sep=',', engine='python')
df_item_properties_part2 = pd.read_csv(filepath_part2, sep=',', engine='python')

df_item_properties = pd.concat(
    [df_item_properties_part1, df_item_properties_part2],
    ignore_index=True
)
display(df_item_properties.head())

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [9]:
print(df_item_properties.columns.tolist())

['timestamp', 'itemid', 'property', 'value']


## Load category_tree.csv

### Subtask:
Load the 'category_tree.csv' file into a pandas DataFrame.

**Reasoning**:
Load the 'category_tree.csv' file into a pandas DataFrame and display the first 5 rows to verify successful loading.



In [10]:
df_category_tree = pd.read_csv(f'{path}/category_tree.csv')
display(df_category_tree.head())

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


## Summary:

### Q&A
*   **Were all specified datasets loaded successfully?**
    Yes, both `item_properties.csv` (by concatenating its two parts) and `category_tree.csv` were loaded successfully into pandas DataFrames.

### Data Analysis Key Findings
*   The `item_properties` dataset was found to be split into two files: `item_properties_part1.csv` and `item_properties_part2.csv`, which were successfully loaded and concatenated into a single DataFrame named `df_item_properties`.
*   The `category_tree.csv` file was successfully loaded into a DataFrame named `df_category_tree`.

### Insights or Next Steps
*   Proceed with initial data exploration and cleaning for `df_item_properties` and `df_category_tree` to understand their structure and content.
*   Consider merging or joining these datasets with other relevant data (if available) for a more comprehensive analysis, depending on the overall project goals.


In [11]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [12]:
df_category_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


In [13]:
df_item_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20275902 entries, 0 to 20275901
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   itemid     int64 
 2   property   object
 3   value      object
dtypes: int64(2), object(2)
memory usage: 618.8+ MB


In [14]:
df_events.isnull().sum()

,0
timestamp,0
visitorid,0
event,0
itemid,0
transactionid,2733644


## Preprocessing

### events.csv

In [15]:
df_events = df_events.dropna()

In [16]:
df_events = df_events.drop_duplicates()

In [17]:
# The time is in unix milliseconds, so converting it into datetime format
df_events['timestamp'] = pd.to_datetime(df_events['timestamp'])

In [18]:
# Remove invalid events (if any)
valid = ['view', 'addtocart', 'transaction']
df_events = df_events[df_events['event'].isin(valid)]

In [19]:
# Remove rows with missing itemid or visitorid
df_events = df_events.dropna(subset=['visitorid', 'itemid'])

In [20]:
df_events = df_events.reset_index(drop=True)

In [21]:
df_events.head(5)

,timestamp,visitorid,event,itemid,transactionid
0,1970-01-01 00:23:53.222276276,599528,transaction,356475,4000.0
1,1970-01-01 00:23:53.193500981,121688,transaction,15335,11117.0
2,1970-01-01 00:23:53.193915008,552148,transaction,81345,5444.0
3,1970-01-01 00:23:53.176736375,102019,transaction,150318,13556.0
4,1970-01-01 00:23:53.174518180,189384,transaction,310791,7244.0


### item_properties.csv

In [22]:
df_item_properties.head(5)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [23]:
df_item_properties.columns

Index(['timestamp', 'itemid', 'property', 'value'], dtype='object')

In [24]:
df_item_properties['timestamp'] = pd.to_datetime(df_item_properties['timestamp'], unit='ms')

In [25]:
df_item_properties['value'] = df_item_properties['value'].astype(str)
df_item_properties['property'] = df_item_properties['property'].astype(str)

In [26]:
df_item_properties.drop_duplicates(inplace=True)

In [27]:
df_item_properties.sort_values(['itemid', 'property', 'timestamp'], inplace=True)

In [28]:
df_item_properties.head(5)

,timestamp,itemid,property,value
13333874,2015-06-07 03:00:00,0,1036,1276750
2419639,2015-05-31 03:00:00,0,1056,n3.168 1144008
19497068,2015-06-07 03:00:00,0,11,n15360.000 628176 n12288.000
140433,2015-06-07 03:00:00,0,112,679677
11373812,2015-05-31 03:00:00,0,127,1168476
